In [214]:
import pandas as pd
from pathlib import Path  
from itertools import product
import numpy as np


In [216]:
# TODO: re add thes

filePath = Path('data/popularmovies.csv')  
df = pd.read_csv(filePath) 
df = df.loc[:, df.columns.intersection(['title','overview', 'popularity'])]
# df.popularity = (df.popularity-df.popularity.min())/(df.popularity.max()-df.popularity.min())
df.popularity = np.log(df.popularity)
df.sort_values('popularity')

,title,overview,popularity
66,Hotarubi no Mori e,One hot summer day a little girl gets lost in ...,-3.649659
17,Grave of the Fireflies,"In the final months of World War II, 14-year-o...",-2.764621
572,Lisbela and the Prisoner,Lisbela is a young woman who loves going to th...,0.219136
1494,Home Alone,Eight-year-old Kevin McCallister makes the mos...,1.342865
191,"“The Shorts” by Aldo, Giovanni and Giacomo","I Corti (""Shorts"") by Aldo, Giovanni & Giacomo...",1.343648
...,...,...,...
920,Deadpool & Wolverine,A listless Wade Wilson toils away in civilian ...,6.375480
46,The Wild Robot,"After a shipwreck, an intelligent robot called...",6.441916
1527,Mufasa: The Lion King,"Mufasa, a cub lost and alone, meets a sympathe...",7.838404
1026,Sonic the Hedgehog 3,"Sonic, Knuckles, and Tails reunite against a p...",8.177634


In [217]:
# clean title row
def clean_title(title):
    words = title.split()
    # if (words[0].lower() == 'the'):
    #     return " ".join(words[1:]).lower()
    return title.lower()

df['clean_title'] = df.title.apply(clean_title)
df['title_len'] = df.title.str.split().apply(len)

colNames = ['title','overview','popularity','clean_title','title_len']
df = df.loc[df.title_len > 1, colNames]

df['first_word'] = df.clean_title.str.split().str[0]
df['last_word'] = df.clean_title.str.split().str[-1]

In [218]:
# Do a bland Movie Centipede with just complete words
# add two columns. One for head and one for tail
lastWords = df.groupby('last_word')['clean_title'].apply(list).to_dict()
firstWords = df.groupby('first_word')['clean_title'].apply(list).to_dict()

df['heads'] = df['last_word'].map(lambda x: firstWords.get(x,[]))
df['tails'] = df['first_word'].map(lambda x: lastWords.get(x,[]))

df['tail_length'] = df['tails'].apply(len)
df['head_length'] = df['heads'].apply(len)
df['naiive_score'] = df.tail_length * df.head_length

df = df.sort_values("naiive_score", ascending = False)
df.loc[df['clean_title'] == 'transformers one']


,title,overview,popularity,clean_title,title_len,first_word,last_word,heads,tails,tail_length,head_length,naiive_score
208,Transformers One,The untold origin story of Optimus Prime and M...,5.772316,transformers one,2,transformers,one,"[one flew over the cuckoo's nest, one hundred ...",[],0,7,0


In [226]:
#for every row

def generate_centipedes(row):
    return list(product(row['tails'], [row['clean_title']], row['heads']))
    
df['full_centipede'] = df.apply(generate_centipedes, axis=1)
df_centipede = df.explode('full_centipede', ignore_index=True)
df_centipede = df_centipede.loc[df_centipede.naiive_score > 0]


df_centipede[['head', 'body', 'tail']] = pd.DataFrame(df_centipede['full_centipede'].tolist(), index=df_centipede.index)
df_centipede = df_centipede.drop(columns =['tail_length',	'head_length','naiive_score', 'heads', 'tails', 'first_word', 'last_word', 'title_len', 'clean_title', 'title'])
df_centipede = df_centipede.rename(columns={"popularity": "body_pop", "overview": "body_view"})
df_centipede = df_centipede[['head', 'body', 'tail',"body_pop", "body_view"]]
df_popularity = df.drop(columns = ['title','overview','title_len','first_word','last_word','heads','tails','tail_length','head_length','naiive_score', 'full_centipede'])

df_centipede = df_centipede.merge(df_popularity, left_on='head', right_on='clean_title', how='left', suffixes=("", "").rename(columns={'popularity': 'head_pop'}).drop(columns = ["clean_title"])
df_centipede = df_centipede.merge(df_popularity, left_on='tail', right_on='clean_title', how='left').rename(columns={'popularity': 'tail_pop'}).drop(columns = ["clean_title"])
# df_centipede = df_centipede.drop(columns = ['full_centipede'])
df_centipede['least_pop'] = df_centipede[['head_pop', 'body_pop', 'tail_pop']].min(axis=1)

# make a new df_for each instance that includes the head, body, tail, head_pop, body_pop, tail_pop

In [227]:
df_centipede.sort_values("least_pop", ascending = False)

# I want to sort by popularity of lowest

,head,body,tail,body_pop,body_view,head_pop,tail_pop,least_pop
270,shaun of the dead,dead poets society,society of the snow,4.229429,"At an elite, old-fashioned boarding school in ...",3.946135,4.073495,3.946135
321,apocalypse now,now you see me,me before you,3.938120,An FBI agent and an Interpol detective track a...,4.044874,4.637608,3.938120
218,puss in boots: the last wish,wish dragon,dragon ball z: bardock - the father of goku,3.750985,Determined teen Din is longing to reconnect wi...,4.966906,3.941096,3.750985
215,puss in boots: the last wish,wish dragon,dragon ball super: super hero,3.750985,Determined teen Din is longing to reconnect wi...,4.966906,3.810898,3.750985
169,the skin i live in,in the mood for love,love don't co$t a thing,3.759431,"In 1962 Hong Kong, two neighbors form a strong...",3.739953,4.137643,3.739953
...,...,...,...,...,...,...,...,...
194,"batman: the long halloween, part one",one piece film: strong world,world of tomorrow,3.013228,"20 years after his escape from Impel Down, the...",2.616666,1.618001,1.618001
362,the skin i live in,in this corner of the world,world of tomorrow,2.429482,"Japan, 1943, during World War II. Young Suzu l...",3.739953,1.618001,1.618001
361,let the right one in,in this corner of the world,world of tomorrow,2.429482,"Japan, 1943, during World War II. Young Suzu l...",3.155766,1.618001,1.618001
175,hair love,love and death,death note relight 1: visions of a god,2.408655,"In czarist Russia, a neurotic soldier and his ...",1.597163,5.330760,1.597163


,title,overview,popularity,clean_title,title_len,first_word,last_word,heads,tails,tail_length,head_length,naiive_score,full_centipede
